In [1]:
import numpy as np
import time
from astropy.io import ascii
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [2]:
def make_R(filename, col_user, col_item, col_rating, fraction = 0.8):
    start_time = time.time()
    '''create R matrix from a file that contains a table of user IDs, item IDs and ratings
    parameters:
        filename: str; the filename of the table
        col_user: str; the column name of user IDs
        col_item: str; the column name of item IDs
        col_rating: str; the column name of the ratings
        fraction: float; the fraction of data to be used for training
    returns:
        rTrain: 2D numpy array; training R matrix in with r_{i, j} is the rating that user i given to item j
        rTest: 2D numpy array; testing R matrix in with r_{i, j} is the rating that user i given to item j'''
    t = ascii.read(filename)
    nData = np.size(t) # total number of data points(ratings)
    nTrain = int(np.rint(fraction * nData)) # number of training data
    nUser = np.max(t[col_user]) # total number of users
    nitem = np.max(t[col_item]) # total number of items
    
    idxData = np.arange(nData) # an array of idx of data
    np.random.shuffle(idxData) # randomize idx
    idxTrain, idxTest = idxData[:nTrain], idxData[nTrain:]
    
    idxUser = t.index_column(col_user)
    idxItem = t.index_column(col_item)
    idxRating = t.index_column(col_rating)
    
    rTrain = np.zeros(shape = (nUser, nitem))
    rTest = np.zeros(shape = (nUser, nitem))
    for nrow in idxTrain:
        i = t[nrow][idxUser] - 1 # the userId
        j = t[nrow][idxItem] - 1 # the itemId
        rTrain[i][j] = t[nrow][idxRating] # the rating
    
    for nrow in idxTest:
        i = t[nrow][idxUser] - 1 # the userId
        j = t[nrow][idxItem] - 1 # the itemId
        rTest[i][j] = t[nrow][idxRating] # the rating
    
    print('the process took %.2f s' % (time.time() - start_time))
    return rTrain, rTest

R_train, R_test = make_R(filename = 'movie_ratings.csv', col_user = 'userId', col_item = 'movieId', col_rating = 'rating')

the process took 2.51 s


In [10]:
class matrix_factorization():

    def __init__(self, R_train, R_test):
        '''R_train: 2D numpy array; R matrix in with r_{i, j} is the rating that user i given to item j'''
        self.R_train = R_train
        self.R_test = R_test
        self.nUser, self.nItem = R_train.shape
        self.result = None

    def train(self, k, alpha, _lambda, iterations):
        '''perform matrix factorization to predict empty entries in a matrix.
        parameters:
            k: int; dimensions of u and v vector
            alpha: float; learning rate
            _lambda: float; regularization parameter'''
        self.k = k
        self.alpha = alpha
        self.iterations = iterations
        self._lambda = _lambda
        
        def update():
            for i, j, r in self.samples:
                # Computer prediction and error
                if _lambda is not None:
                    prediction = self.b + self.b_u[i] + self.b_i[j] + self.U[i, :].dot(self.V[j, :].T)
                    e = (r - prediction)
                    # Update biases
                    self.b_u[i] += self.alpha * (e - self._lambda * self.b_u[i])
                    self.b_i[j] += self.alpha * (e - self._lambda * self.b_i[j])

                    # Update user and item latent feature matrices
                    self.U[i, :] += self.alpha * (e * self.V[j, :] - self._lambda * self.U[i,:])
                    self.V[j, :] += self.alpha * (e * self.U[i, :] - self._lambda * self.V[j,:])
                else:
                    prediction = self.U[i, :].dot(self.V[j, :].T)
                    e = (r - prediction)
                    # Update user and item latent feature matrices
                    self.U[i, :] += self.alpha * e * self.V[j, :]
                    self.V[j, :] += self.alpha * e * self.U[i, :]
                    
        start_time = time.time()
        # Initialize user and item latent feature matrice
        self.U = np.random.normal(scale = 1./self.k, size = (self.nUser, self.k))
        self.V = np.random.normal(scale = 1./self.k, size = (self.nItem, self.k))
        
        if _lambda is not None:
            # Initialize the biases
            self.b_u = np.zeros(self.nUser)
            self.b_i = np.zeros(self.nItem)
            self.b = np.mean(self.R_train[np.where(self.R_train != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R_train[i, j])
            for i in range(self.nUser)
            for j in range(self.nItem)
            if self.R_train[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            update()
                
        if _lambda is not None:
            self.result = self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.U.dot(self.V.T)
        else:
            self.result = self.U.dot(self.V.T)
        
        sigma_train, sigma_test = self.sigma()
        print('training is complete! it took %.2f s' % (time.time() - start_time))
        print("train error = %.4f; test error = %.4f" % (sigma_train, sigma_test))
        return self.result
    
    def sigma(self):
        '''Compute the total mean square error for training and testing data'''
        iTrain, jTrain = self.R_train.nonzero()
        iTest, jTest = self.R_test.nonzero()
        trainError, testError = 0, 0
        trainCount, testCount = 0, 0
        for i, j in zip(iTrain, jTrain):
            trainError += (self.R_train[i, j] - self.result[i, j])**2
            trainCount += 1
            
        for i, j in zip(iTest, jTest):
            testError += (self.R_test[i, j] - self.result[i, j])**2
            testCount += 1
        
        trainSigma = np.sqrt(trainError)/trainCount
        testSigma = np.sqrt(testError)/testCount
        return trainSigma, testSigma

In [14]:
class correlation_similarity():
    def __init__(self, R_train, R_test):
        '''initialize
        R_train: 2D numpy array; R matrix in with r_{i, j} is the rating that user i given to item j
        R_test: 2D numpy array; R matrix in with r_{i, j} is the rating that user i given to item j'''
        self.R_train = R_train
        self.R_test = R_test
        self.nUser, self.nItem = R_train.shape
        self.result = None
        self.S = cosine_similarity(sparse.csr_matrix(R_train)) # similarity 
        self.avg = np.mean(R_train)
        
    def predict(self, i, j, k):
        '''predict the rating for user i of item j
        i: int; the user number
        j: int; the item number
        k: int; use k nearest neighbors'''
        if self.R_train[i][j] != 0:
            return self.R_train[i][j]
        else:
            idxS_user = np.argsort(self.S[i])
            idx_n = np.where(self.R_train[:, j] > 0)[0]

            if np.size(idx_n) < k:
                if np.size(idx_n) == 0: 
#                     idxS_item = np.argsort(self.S_item[j]) # similarity between items
                    v_j = self.R_train[:, j]
                    prediction = v_j[v_j != 0].mean()
                else:
                    prediction = self.R_train[idx_n, j].mean()
            else:
                idx_knn = idxS_user[np.isin(idxS_user, idx_n, assume_unique = True)][-k - 1:-1] # idxS with ratings
                prediction = self.R_train[idx_knn, j].mean()
            if np.isnan(prediction):
                prediction = self.avg
            return prediction

    def sigma2(self, i, j, k):
        if self.R_test[i][j] == 0:
            print('No true value!')
            return None
        else:
            r_hat = self.predict(i, j, k)
            return (self.R_test[i][j] - r_hat)**2
        
    def test(self, k):
        start_time = time.time()
        iTest, jTest = self.R_test.nonzero()
        totalError = 0
        testCount = 0
        for i, j in zip(iTest, jTest):
            totalError += self.sigma2(i, j, k)
            testCount += 1
        sigma = np.sqrt(totalError)/testCount         
        print('testing is complete! it took %.2f s' % (time.time() - start_time))
        print('the test error is %.4f' % sigma)
        return sigma

In [15]:
rs = correlation_similarity(R_train, R_test)
rs.test(3)

/home/bluesky/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: Mean of empty slice.
/home/bluesky/.local/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


testing is complete! it took 1.83 s
the test error is 0.0087


0.008653818512465552

In [ ]:
a = np.nan
a == np.nan

In [11]:
mf = matrix_factorization(R_train, R_test)
# mf.correlation_similarity(3)
mf.train(k = 300, alpha = 0.01, _lambda = 0.01, iterations = 20)

training is complete! it took 46.47 s
train error = 0.0025; test error = 0.0060


array([[4.61381331, 4.16390999, 3.96063549, ..., 4.18826672, 4.18621446,
        4.30223442],
       [3.91904793, 3.52800818, 3.25229948, ..., 3.52446749, 3.52422589,
        3.63747933],
       [2.523302  , 2.27027375, 1.91053794, ..., 2.21544257, 2.21769012,
        2.32679888],
       ...,
       [2.69961665, 2.57909417, 2.34110182, ..., 3.09949476, 3.10133321,
        3.1952866 ],
       [3.63104007, 3.24922679, 2.97311836, ..., 3.25355858, 3.25410396,
        3.36804408],
       [4.24056327, 3.38398571, 3.3739145 , ..., 3.52643077, 3.52222999,
        3.66044745]])

In [ ]:
mf.mse()

In [ ]:
tR = np.array([
    [5, 3, 0, 1, 1],
    [4, 0, 0, 1, 1],
    [1, 1, 0, 5, 4],
    [1, 0, 0, 4, 5],
    [0, 1, 5, 4, 4],
#     [0, 1, 5, 4, 4],
])

In [ ]:
cosine_similarity(tR)

In [ ]:
cosine_similarity(tR.T)

In [ ]:
1 - spatial.distance.cosine(tR[0], tR[5])

In [ ]:
correlation_similarity(tR, 3)

In [ ]:
tR_sparse = sparse.csr_matrix(tR)

In [ ]:
h = np.array([0, 1, 2])
tR[4][h]

In [ ]:
similarities

In [ ]:
from scipy import spatial

In [ ]:
1 - spatial.distance.cosine(tR[0], tR[2])

In [ ]:
R_train

In [ ]:
np.arange(100)

In [ ]:
x = np.arange(100)
np.random.shuffle(x)
training, test = x[:80], x[80:]

In [ ]:
training

In [ ]:
def mock_R(nU, nI, f = 0.2, ft = 0.8):
    R = np.zeros(shape = (nU, nI))
    n = int(np.rint(f * nU * nI))
    idxU = np.random.choice(np.arange(nU), size = n)
    idxI = np.random.choice(np.arange(nI), size = n)
    for x, y in zip(idxU, idxI):
        R[x][y] = np.random.randint(low = 1, high = 10)
    return R

In [ ]:
mR = mock_R(40, 50)
mR

In [ ]:
mmR = recommendation_system(mR, mR)

In [ ]:
mmR.matrix_factorization(k = 300, alpha = 0.01, _lambda = 0.01, iterations = 20)